## Loading JSON

In [1]:
import pandas as pd
import json

def load_data(location):
    f = open(location)
    data = json.load(f)
    
    arr = []
    
    for key in data:
        temp = []

        if data[key]['edges'] == False:
            temp.append(0)
        else:
            temp.append(1)
        if data[key]['magnitude'] == False:
            temp.append(0)
        else:
            temp.append(1)
        temp.append(data[key]['cascadeClassifier']['bottom_left'])
        temp.append(data[key]['cascadeClassifier']['bottom_right'])
        
        arr.append(temp)

    return pd.DataFrame(arr, columns=['edges', 'magnitude', 'quadrant3', 'quadrant4'])


data0001 = load_data('./jsons/2011_09_26_drive_0001_extract.json')
data0002 = load_data('./jsons/2011_09_26_drive_0002_extract.json')
data0013 = load_data('./jsons/2011_09_26_drive_0013_extract.json')
data0017 = load_data('./jsons/2011_09_26_drive_0017_extract.json')
data0048 = load_data('./jsons/2011_09_26_drive_0048_extract.json')
data0060 = load_data('./jsons/2011_09_26_drive_0060_extract.json')
data0113 = load_data('./jsons/2011_09_26_drive_0113_extract.json')
data0026 = load_data('./jsons/2011_09_29_drive_0026_extract.json')

print(data0017)

     edges  magnitude  quadrant3  quadrant4
0        0          0          0          0
1        0          0          0          0
2        0          0          0          0
3        0          0          0          0
4        0          0          0          0
..     ...        ...        ...        ...
115      0          0          0          0
116      0          1          0          0
117      0          1          0          0
118      0          1          0          0
119      0          1          0          0

[120 rows x 4 columns]


## Loading labeld results

In [2]:
label0001 = pd.read_csv('./utils/2011_09_26_drive_0001_extract.csv')
label0002 = pd.read_csv('./utils/2011_09_26_drive_0002_extract.csv')
label0013 = pd.read_csv('./utils/2011_09_26_drive_0013_extract.csv')
label0017 = pd.read_csv('./utils/2011_09_26_drive_0017_extract.csv')
label0048 = pd.read_csv('./utils/2011_09_26_drive_0048_extract.csv')
label0060 = pd.read_csv('./utils/2011_09_26_drive_0060_extract.csv')
label0113 = pd.read_csv('./utils/2011_09_26_drive_0113_extract.csv')
label0026 = pd.read_csv('./utils/2011_09_29_drive_0026_extract.csv')

print(label0017)

             frames  safety
0    0000000000.png       0
1    0000000001.png       0
2    0000000002.png       0
3    0000000003.png       0
4    0000000004.png       0
..              ...     ...
115  0000000115.png       0
116  0000000116.png       0
117  0000000117.png       0
118  0000000118.png       0
119  0000000119.png       0

[120 rows x 2 columns]


---

## Trainning

In [3]:
from sklearn.neural_network import MLPClassifier

In [4]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

Xs = [data0001, data0002, data0013, data0017, data0060, data0026, data0113] #Leave data0048 for checking accuracy
ys = [label0001, label0002, label0013, label0017, label0060, label0026, label0113] #Leave label0048 for checking accuracy

for i in range(len(Xs)):
    print(i)
    X = Xs[i]
    y = ys[i]['safety']
    clf.fit(X, y)

0
1
2
3
4
5
6


Save the trained model.

In [5]:
import pickle

with open('model.pkl','wb') as f:
    pickle.dump(clf, f)

## Results

In [6]:
X = data0048

y_act = label0048['safety']
y_pred = clf.predict(X)

In [7]:
from prettytable import PrettyTable

def calculate_metrics(true_labels, pred_labels):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for i in range(len(true_labels)):
        if true_labels[i] == pred_labels[i]:
            if true_labels[i] == 1:
                tp += 1
            else:
                tn += 1
        else:
            if true_labels[i] == 1:
                fn += 1
            else:
                fp += 1
    
    accuracy = (tp + tn) / len(true_labels)
    false_positive_rate = fp / (fp + tn)
    false_negative_rate = fn / (fn + tp)
    
    # create a table to display the results
    x = PrettyTable()
    x.field_names = ["Metric", "Value"]
    x.add_row(["Accuracy", "{:.2f}".format(accuracy)])
    x.add_row(["False Positive Rate", "{:.2f}".format(false_positive_rate)])
    x.add_row(["False Negative Rate", "{:.2f}".format(false_negative_rate)])
    
    print(x)

In [8]:
calculate_metrics(y_act, y_pred)

+---------------------+-------+
|        Metric       | Value |
+---------------------+-------+
|       Accuracy      |  0.64 |
| False Positive Rate |  0.38 |
| False Negative Rate |  0.33 |
+---------------------+-------+


## Output results

In [9]:
import os
import json

def gen_final_results():
    # Create results folder if not exist
    if not os.path.exists("results"):
        os.makedirs("results")
    
    json_folder = 'jsons'
    for file_name in os.listdir(json_folder):
        file_path = os.path.join(json_folder, file_name)
    
        # Get result from CLF
        data = load_data(file_path)
        result = clf.predict(data)
    
        # Compose the result JSON
        json_to_write = {}
        index = 0

        f = open(file_path)
        original_data = json.load(f)
        for key in original_data:
            result_boolean = True
            if result[index] == 0:
                result_boolean = False
            json_to_write[key] = result_boolean
            index += 1
            
        with open(os.path.join("results", file_name), 'w') as file_to_write:
            json.dump(json_to_write, file_to_write)
        

gen_final_results()